<a href="https://colab.research.google.com/github/Tgreen289/Use-GFPGAN-for-large-images-Colab-/blob/main/GFPGAN_inference_largeimgsupport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Preparations
Before start, make sure that you choose
* Runtime Type = Python 3
* Hardware Accelerator = GPU

Run on CPU is possible but Interfere will be slower.
In the **Runtime** menu -> **Change runtime type**

Then, we clone the repository, set up the environment, and download the pre-trained model.


In [ ]:
# Clone GFPGAN and enter the GFPGAN folder
%cd /content
!rm -rf GFPGAN
!git clone https://github.com/TencentARC/GFPGAN.git
%cd GFPGAN

# Set up the environment
# Install basicsr - https://github.com/xinntao/BasicSR
# We use BasicSR for both training and inference
!pip install basicsr
# Install facexlib - https://github.com/xinntao/facexlib
# We use face detection and face restoration helper in the facexlib package
!pip install facexlib
# Install other depencencies
!pip install -r requirements.txt
!python setup.py develop
!pip install realesrgan  # used for enhancing the background (non-face) regions
# Download the pre-trained model
# !wget https://github.com/TencentARC/GFPGAN/releases/download/v0.2.0/GFPGANCleanv1-NoCE-C2.pth -P experiments/pretrained_models
# Now we use the V1.4 model for the demo
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth -P experiments/pretrained_models


/content
Cloning into 'GFPGAN'...
remote: Enumerating objects: 523, done.
remote: Counting objects: 100% (210/210), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 523 (delta 168), reused 153 (delta 153), pack-reused 313
Receiving objects: 100% (523/523), 5.37 MiB | 8.06 MiB/s, done.
Resolving deltas: 100% (279/279), done.
/content/GFPGAN
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.0 MB/s eta 0:00:00
  Created wheel for basicsr: filename=basicsr-1.4.2-py3-none-any.whl size=214819 sha256=3568b91d19f08de138861ed4777bd263ed085c4a46cd2b022fccab465d14e1e9
  Stored in directory: /root/.cache/pip

In [ ]:
# @title Change crop ratio(any image with width > 1024 needs CR = image width/1024 ) { run: "auto" }
CR = 1.6 # @param ["1", "1.3", "1.6", 1.8 , "2", "3.2", "4", "6.4", "8", "16"] {type:"raw", allow-input: true}
CR = float(CR)
FS = 512/CR
# Read in the file
with open("/content/GFPGAN/gfpgan/utils.py", "r") as file :
 data = file.readlines()
# Replace the target string
 data[80] = "            face_size="+str(FS)+ ",\n"
 data[81] = "            crop_ratio=("+str(CR)+", "+str(CR)+"),\n"
# Write the file out
with open("/content/GFPGAN/gfpgan/utils.py", 'w') as file:
  file.writelines(data)

In [ ]:
# @title Change output to only restored images + comparison
with open("/content/GFPGAN/inference_gfpgan.py", "r") as file :
 data = file.readlines()
# Replace the target string
 data[143] = "             #save_crop_path = os.path.join(args.output, 'cropped_faces', f'{basename}_{idx:02d}.png')\n"
 data[144] = "             #imwrite(cropped_face, save_crop_path)\n"
 data[146] = "             if args.suffix is not None:\n"
 data[147] = "                save_face_name = f'{basename}_{idx:02d}_{args.suffix}.png'\n"
 data[148] = "             else:\n"
 data[149] = "                save_face_name = f'{basename}_{idx:02d}.png'\n"
 data[150] = "             save_restore_path = os.path.join(args.output, 'restored_faces', save_face_name)\n"
 data[151] = "             imwrite(restored_face, save_restore_path)\n"
 data[153] = "             cmp_img = np.concatenate((cropped_face, restored_face), axis=1)\n"
 data[154] = "             imwrite(cmp_img, os.path.join(args.output, 'cmp', f'{basename}_{idx:02d}.png'))\n"
# Write the file out
with open("/content/GFPGAN/inference_gfpgan.py", 'w') as file:
  file.writelines(data)

In [ ]:
# @title Change output to only restored images(this will disable Visualize)
with open("/content/GFPGAN/inference_gfpgan.py", "r") as file :
 data = file.readlines()
# Replace the target string
 data[143] = "             #save_crop_path = os.path.join(args.output, 'cropped_faces', f'{basename}_{idx:02d}.png')\n"
 data[144] = "             #imwrite(cropped_face, save_crop_path)\n"
 data[146] = "             if args.suffix is not None:\n"
 data[147] = "                save_face_name = f'{basename}_{idx:02d}_{args.suffix}.png'\n"
 data[148] = "             else:\n"
 data[149] = "                save_face_name = f'{basename}_{idx:02d}.png'\n"
 data[150] = "             #save_restore_path = os.path.join(args.output, 'restored_faces', save_face_name)\n"
 data[151] = "             #imwrite(restored_face, save_restore_path)\n"
 data[153] = "             #cmp_img = np.concatenate((cropped_face, restored_face), axis=1)\n"
 data[154] = "             #imwrite(cmp_img, os.path.join(args.output, 'cmp', f'{basename}_{idx:02d}.png'))\n"
# Write the file out
with open("/content/GFPGAN/inference_gfpgan.py", 'w') as file:
  file.writelines(data)

In [ ]:
# @title Change output back to default(cmp+crop+restored_img)
with open("/content/GFPGAN/inference_gfpgan.py", "r") as file :
 data = file.readlines()
# Replace the target string
 data[143] = "             save_crop_path = os.path.join(args.output, 'cropped_faces', f'{basename}_{idx:02d}.png')\n"
 data[144] = "             imwrite(cropped_face, save_crop_path)\n"
 data[146] = "             if args.suffix is not None:\n"
 data[147] = "                save_face_name = f'{basename}_{idx:02d}_{args.suffix}.png'\n"
 data[148] = "             else:\n"
 data[149] = "                save_face_name = f'{basename}_{idx:02d}.png'\n"
 data[150] = "             save_restore_path = os.path.join(args.output, 'restored_faces', save_face_name)\n"
 data[151] = "             imwrite(restored_face, save_restore_path)\n"
 data[153] = "             cmp_img = np.concatenate((cropped_face, restored_face), axis=1)\n"
 data[154] = "             imwrite(cmp_img, os.path.join(args.output, 'cmp', f'{basename}_{idx:02d}.png'))\n"
# Write the file out
with open("/content/GFPGAN/inference_gfpgan.py", 'w') as file:
  file.writelines(data)

# 2. Upload Images / Use the demo images

In [ ]:
# upload your own images
import os
from google.colab import files
import shutil

upload_folder = 'inputs/upload'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
os.mkdir(upload_folder)

# upload images
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

Saving me.jpg to me.jpg
move me.jpg to inputs/upload/me.jpg


### OR you can use the demo image by running the following codes

In [ ]:
import shutil
import os
upload_folder = 'inputs/upload'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
os.makedirs(upload_folder, exist_ok=True)
shutil.move('inputs/whole_imgs/Blake_Lively.jpg', 'inputs/upload/Blake_Lively.jpg')

## 3. Inference

In [ ]:
# Now we use the GFPGAN to restore the above low-quality images
# We use [Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN) for enhancing the background (non-face) regions
# You can find the different models in https://github.com/TencentARC/GFPGAN#european_castle-model-zoo
!rm -rf results
!python inference_gfpgan.py -i inputs/upload -o results -v 1.4 -s 1 --bg_upsampler realesrgan

# Usage: python inference_gfpgan.py -i inputs/whole_imgs -o results -v 1.3 -s 2 [options]...
#
#  -h                   show this help
#  -i input             Input image or folder. Default: inputs/whole_imgs
#  -o output            Output folder. Default: results
#  -v version           GFPGAN model version. Option: 1 | 1.2 | 1.3. Default: 1.3
#  -s upscale           The final upsampling scale of the image. Default: 2
#  -bg_upsampler        background upsampler. Default: realesrgan
#  -bg_tile             Tile size for background sampler, 0 for no tile during testing. Default: 400
#  -suffix              Suffix of the restored faces
#  -only_center_face    Only restore the center face
#  -aligned             Input are aligned faces
#  -ext                 Image extension. Options: auto | jpg | png, auto means using the same extension as inputs. Default: auto

!ls results/cmp

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
Downloading: "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth" to /usr/local/lib/python3.10/dist-packages/weights/RealESRGAN_x2plus.pth

100% 64.0M/64.0M [00:01<00:00, 64.9MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the f

## 4. Visualize

In [ ]:
# @title Original crop vs fixed crop image
# We first visualize the cropped faces
# The left are the inputs images; the right are the results of GFPGAN

import cv2
import matplotlib.pyplot as plt
def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1)
  plt.title('Input image', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('GFPGAN output', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

# display each image in the upload folder
import os
import glob

input_folder = 'results/cropped_faces'
result_folder = 'results/restored_faces'
input_list = sorted(glob.glob(os.path.join(input_folder, '*')))
output_list = sorted(glob.glob(os.path.join(result_folder, '*')))
for input_path, output_path in zip(input_list, output_list):
  img_input = imread(input_path)
  img_output = imread(output_path)
  display(img_input, img_output)


In [ ]:
# @title Original image vs restored full image
# We then visualize the whole image
# The left are the inputs images; the right are the results of GFPGAN

import cv2
import matplotlib.pyplot as plt
def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1)
  plt.title('Input image', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('GFPGAN output', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

# display each image in the upload folder
import os
import glob

input_folder = 'inputs/upload'
result_folder = 'results/restored_imgs'
input_list = sorted(glob.glob(os.path.join(input_folder, '*')))
output_list = sorted(glob.glob(os.path.join(result_folder, '*')))
for input_path, output_path in zip(input_list, output_list):
  img_input = imread(input_path)
  img_output = imread(output_path)
  display(img_input, img_output)

## 5. Download results

In [ ]:
# download the result
!ls results
print('Download results')
os.system('zip -r download.zip results')
files.download("download.zip")

cmp  cropped_faces  restored_faces  restored_imgs
Download results


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>